In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# update working directory
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom')

In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
import torch
import numpy as np
import random
import os
import shutil
import re

from parameters import Parameters
from deepfake_datasets import LOCALIZATION_DATASET_PATHS, DETECTION_DATASET_PATHS
from deepfake_datasets.datasets import get_dataloader
from models.declip import get_model
from models.declip_detection import get_detection_model
from train.validate import validate_detection, validate_fully_supervised_localization, validate_fully_supervised_localization_real_images
from utils.utils import compute_mean_iou, compute_mean_f1, compute_mean_ap, compute_mean_acc_detection, compute_mean_ap_detection

In [ ]:
# set seed for reproducibility
SEED = 0
def set_seed():
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

# save image predictions to files
def save_image_localization_scores_to_file(ious, f1_best, f1_fixed, aps, img_paths, params, dataset_name):
    with open(params.save_dir_results + f"/scores_{dataset_name}.txt", 'a') as f:
        f.write(f'image path \t iou \t f1_best \t f1_fixed \t ap\n')
        for iou, f1_b, f1_f, ap, img_path in zip(ious, f1_best, f1_fixed, aps, img_paths):
            f.write(f'{img_path} \t {iou} \t {f1_b} \t {f1_f} \t {ap}\n')

In [ ]:
# set parameters
params = Parameters()

# set the experiment name and output directory
# alert if the experiment name is not set or the output directory already exists
params.experiment_name = 'test_initial_training'
assert params.experiment_name != '', 'Please set the experiment name'
experiment_path = f'/content/experiments/{params.experiment_name}'
if os.path.exists(experiment_path):
    shutil.rmtree(experiment_path)
params.create_output_dirs()

# set data labels to test
params.data_label = 'test'

# set the model checkpoint path
params.checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/combined_training/test_ldm/finetune_ldm/'
assert params.checkpoint_path != '', 'Please set the checkpoint path'

# set model parameters for testing
# state_dict = torch.load(params.checkpoint_path, map_location='cpu')
# params.decoder_type = state_dict['decoder_type']
# params.feature_layer = state_dict['feature_layer']

params.arch = 'CLIP:ViT-L/14,RN50'
# params.arch = 'CLIP:RN50'

params.task_type = 'fully_supervised_localization'

# params.test_real_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets/dolos_data/celebahq/'

# set the batch size and num threads
params.batch_size = 64
params.num_threads = 8

In [ ]:
# prepare the datasets and the results file
if params.task_type == 'fully_supervised_localization' or params.task_type == 'fully_supervised_localization_real_images':
    dataset_paths = LOCALIZATION_DATASET_PATHS
    with open(os.path.join(params.save_dir_results, 'scores.txt'), 'a') as f:
        f.write('dataset \t iou \t f1_best \t f1_fixed \t ap \n')
elif params.task_type == 'detection':
    dataset_paths = DETECTION_DATASET_PATHS
    with open(os.path.join(params.save_dir_results, 'scores.txt'), 'a') as f:
        f.write('dataset \t ap \t acc_fixed_thresh \t acc_best_thresh \t best_threshold \n')

In [ ]:
# load the models
finetuned_models_dir_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/combined_training/test_lama/finetune_lama/'
sorted_files = sorted(os.listdir(finetuned_models_dir_path), key=lambda f: int(re.search(r'round_(\d+)', f).group(1)))
for file_name in sorted_files:
    if file_name.endswith('.pth'):
        params.checkpoint_path = os.path.join(finetuned_models_dir_path, file_name)
        state_dict = torch.load(params.checkpoint_path, map_location='cpu')
        params.decoder_type = state_dict['decoder_type']
        params.feature_layer = state_dict['feature_layer']
        model = get_model(params)
        model.load_state_dict(state_dict['model'], strict=False)
        model = model.cuda()
        model.eval()
        print(f'Loaded model from {file_name}')

        # start the testing process
        for dataset_path in dataset_paths:
            if 'autosplice' in dataset_path['key']:
                continue
            print(f"Testing on {dataset_path['key']}")
            set_seed()
            os.makedirs(os.path.join(params.save_dir_results, dataset_path['key']), exist_ok=True)

            params.train_dataset = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['key'])
            params.test_fake_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['fake_path'])
            params.test_masks_ground_truth_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['masks_path'])
            if params.task_type == 'detection' or  params.task_type == 'fully_supervised_localization_real_images' or params.task_type == 'classification':
                params.test_real_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/', dataset_path['real_path'])

            data_loader = get_dataloader(params)

            # create the directory for dataset results
            os.makedirs(os.path.join(params.save_dir_results, params.train_dataset), exist_ok=True)

            if params.task_type == 'fully_supervised_localization':
                ious, f1_best, f1_fixed, ap, original_img_paths = validate_fully_supervised_localization(
                    model, data_loader, dataset_path['key'], params.save_dir_results
                )
                save_image_localization_scores_to_file(ious, f1_best, f1_fixed, ap, original_img_paths, params, dataset_path['key'])

                mean_iou = compute_mean_iou(ious, verbose=True, extra_text=dataset_path['key'])
                mean_f1_best, mean_f1_fixed = compute_mean_f1(f1_best, f1_fixed, verbose=True, extra_text=dataset_path['key'])
                mean_ap = compute_mean_ap(ap, verbose=True, extra_text=dataset_path['key'])

                with open(os.path.join(params.save_dir_results, f"scores_{dataset_path['key']}.txt"), 'a') as f:
                    f.write(f"{dataset_path['key']} \t {mean_iou:.4f} \t {mean_f1_best:.4f} \t {mean_f1_fixed:.4f} \t {mean_ap:.4f} \n")

            print()

100%|███████████████████████████████████████| 890M/890M [00:13<00:00, 67.4MiB/s]
100%|███████████████████████████████████████| 244M/244M [00:03<00:00, 82.7MiB/s]


Loaded model from test_lama_round_1_best_localization_model_iou_78.3713.pth
Testing on lama
Length of dataset:  900


100%|██████████| 15/15 [02:30<00:00, 10.01s/it]


lamaMean IOU: 79.06
lamaMean F1 best: 0.9229
lamaMean F1 fixed: 0.8875
lamaMean AP: 0.9349

Testing on ldm
Length of dataset:  900


100%|██████████| 15/15 [02:22<00:00,  9.52s/it]


...

ldmMean IOU: 41.6
ldmMean F1 best: 0.6005
ldmMean F1 fixed: 0.4946
ldmMean AP: 0.5997

Testing on pluralistic
Length of dataset:  900


100%|██████████| 15/15 [00:38<00:00,  2.55s/it]


pluralisticMean IOU: 66.64
pluralisticMean F1 best: 0.8572
pluralisticMean F1 fixed: 0.7671
pluralisticMean AP: 0.8496

Testing on repaint-p2-9k
Length of dataset:  900


100%|██████████| 15/15 [00:38<00:00,  2.56s/it]

repaint-p2-9kMean IOU: 56.94
repaint-p2-9kMean F1 best: 0.761
repaint-p2-9kMean F1 fixed: 0.6779
repaint-p2-9kMean AP: 0.7563



In [ ]:
import shutil
from google.colab import files

# Arhivare folder
shutil.make_archive(f'results_{params.checkpoint_path}', 'zip', '/content/experiments/test_initial_training/results')

# Descărcare
files.download(f'results_{params.checkpoint_path}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>